# Missed Call Exploration for MLB Hitters 2023

## Finding Change in Run Expectancy for each Base-Out-Count-Pitch

In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install --upgrade pip setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pynacl

In [4]:
!pip install pybaseball

In [5]:
from pybaseball import statcast
from pybaseball import statcast_batter

In [6]:
from pybaseball import cache

cache.enable()

In [7]:
df_ids = pd.read_csv('player_ids.csv')

In [8]:
df_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 44 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IDPLAYER        3138 non-null   object 
 1   PLAYERNAME      3138 non-null   object 
 2   BIRTHDATE       3138 non-null   object 
 3   FIRSTNAME       3138 non-null   object 
 4   LASTNAME        3138 non-null   object 
 5   TEAM            2077 non-null   object 
 6   LG              2076 non-null   object 
 7   POS             3138 non-null   object 
 8   IDFANGRAPHS     3137 non-null   object 
 9   FANGRAPHSNAME   3138 non-null   object 
 10  MLBID           3137 non-null   float64
 11  MLBNAME         3137 non-null   object 
 12  CBSID           3095 non-null   float64
 13  CBSNAME         3138 non-null   object 
 14  RETROID         2228 non-null   object 
 15  BREFID          3136 non-null   object 
 16  NFBCID          2323 non-null   float64
 17  NFBCNAME        3134 non-null   o

In [9]:
df_ids = df_ids[['MLBID','MLBNAME']]

In [10]:
df_ids = df_ids.dropna()

In [11]:
df_ids['MLBID'] = df_ids['MLBID'].astype(int)

In [12]:
df = statcast(start_dt="2023-03-30", end_dt="2023-08-14",verbose=True)

This is a large query, it may take a moment to complete


100%|██████████| 138/138 [00:06<00:00, 22.01it/s]


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526003 entries, 1953 to 4406
Data columns (total 92 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   pitch_type                       525706 non-null  object        
 1   game_date                        526003 non-null  datetime64[ns]
 2   release_speed                    525684 non-null  float64       
 3   release_pos_x                    525704 non-null  float64       
 4   release_pos_z                    525704 non-null  float64       
 5   player_name                      526003 non-null  object        
 6   batter                           526003 non-null  Int64         
 7   pitcher                          526003 non-null  Int64         
 8   events                           134829 non-null  object        
 9   description                      526003 non-null  object        
 10  spin_dir                         0 non-null

In [14]:
df = df.merge(df_ids,left_on='batter',right_on='MLBID')

In [15]:
df = df.drop('player_name',axis=1)

In [16]:
df = df.rename(columns={'MLBNAME':'player_name'})

In [17]:
df = df.loc[(df['description'] == 'called_strike') | (df['description'] == 'ball')]

In [18]:
df = df[df['delta_run_exp'].notna()]

In [19]:
df = df[df['zone'].notna()]

In [20]:
def batter_team(df):
    if df['inning_topbot'] == 'Top':
        return df['away_team']
    else:
        return df['home_team']

In [21]:
df['batter_team'] = df.apply(batter_team,axis=1)

In [22]:
df['player_name'] = df['player_name'] + '-' + df['batter_team']

In [23]:
df['events'].value_counts()

walk                          10334
strikeout                      6945
caught_stealing_2b              133
strikeout_double_play            17
pickoff_1b                        7
caught_stealing_home              6
wild_pitch                        5
pickoff_caught_stealing_2b        5
caught_stealing_3b                4
other_out                         3
pickoff_3b                        2
catcher_interf                    1
Name: events, dtype: int64

In [24]:
df = df.loc[(df['events']=='walk') | (df['events']=='strikeout') | (df['events'].isna())]

In [25]:
df['description'].value_counts()

ball             175058
called_strike     86086
Name: description, dtype: int64

In [26]:
df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)

In [27]:
df['on_base'] = np.where(df['on_1b'].notna(),'1','_')

In [28]:
df['on_base'] = df['on_base'] + '-' + np.where(df['on_2b'].notna(),'2','_')

In [29]:
df['on_base'] = df['on_base'] + '-' + np.where(df['on_3b'].notna(),'3','_')

In [30]:
df_all_calls = df.groupby(by=['player_name','description'])['pitch_type'].count().reset_index().pivot_table('pitch_type',['player_name'],'description').reset_index()

In [31]:
df_all_calls.head(20)

description,player_name,ball,called_strike
0,A.J. Pollock-SEA,169.0,73.0
1,A.J. Pollock-SF,13.0,5.0
2,Aaron Hicks-BAL,247.0,141.0
3,Aaron Hicks-NYY,112.0,53.0
4,Aaron Judge-NYY,459.0,190.0
5,Abraham Almonte-NYM,11.0,6.0
6,Abraham Toro-MIL,17.0,18.0
7,Adam Duvall-BOS,260.0,153.0
8,Adam Engel-SD,8.0,4.0
9,Adam Frazier-BAL,443.0,252.0


In [32]:
df['pa'] = df['game_pk'].astype(str) + '-' + df['at_bat_number'].astype(str)

In [33]:
df_grouped = df.groupby(by=['on_base','outs_when_up','count','type'])['delta_run_exp'].agg(pd.Series.mode).to_frame().reset_index()

In [34]:
df_grouped.head()

,on_base,outs_when_up,count,type,delta_run_exp
0,1-2-3,0,0-0,B,0.101
1,1-2-3,0,0-0,S,-0.123
2,1-2-3,0,0-1,B,0.101
3,1-2-3,0,0-1,S,-0.147
4,1-2-3,0,0-2,B,0.119


## Finding All Missed Calls

In [35]:
def actual_strike(df):
    if df['zone'] > 10:
        return 'B'
    else:
        return 'S'

In [36]:
df['correct_type'] = df.apply(actual_strike,axis=1)

In [37]:
df_merged = df.merge(df_grouped,left_on=['on_base','outs_when_up','count','correct_type'],right_on=['on_base','outs_when_up','count','type'])

In [38]:
df_merged['missed_call_delta_run_exp'] = df_merged['delta_run_exp_x'] - df_merged['delta_run_exp_y']

In [39]:
df_merged['missed_call_delta_run_exp'].sum()

-188.21299999999997

In [40]:
df_mlb = df_merged[df_merged['missed_call_delta_run_exp']!=0].groupby(by=['player_name','correct_type']).count().reset_index().iloc[0:,0:3]

In [41]:
df_mlb.head()

,player_name,correct_type,pitch_type
0,A.J. Pollock-SEA,B,9
1,A.J. Pollock-SEA,S,7
2,A.J. Pollock-SF,B,1
3,A.J. Pollock-SF,S,2
4,Aaron Hicks-BAL,B,13


In [42]:
df_mlb_pivoted = df_mlb.pivot_table('pitch_type', ['player_name'], 'correct_type')

In [43]:
df_mlb_pivoted.reset_index(inplace=True)

In [44]:
df_mlb_pivoted.head()

correct_type,player_name,B,S
0,A.J. Pollock-SEA,9.0,7.0
1,A.J. Pollock-SF,1.0,2.0
2,Aaron Hicks-BAL,13.0,9.0
3,Aaron Hicks-NYY,11.0,6.0
4,Aaron Judge-NYY,34.0,21.0


In [45]:
df_mlb_pivoted.fillna(0,inplace=True)

In [46]:
df_missed_and_total = df_mlb_pivoted.merge(df_all_calls,left_on='player_name',right_on='player_name')

In [47]:
df_missed_and_total.fillna(0,inplace=True)

In [48]:
df_missed_and_total.head()

,player_name,B,S,ball,called_strike
0,A.J. Pollock-SEA,9.0,7.0,169.0,73.0
1,A.J. Pollock-SF,1.0,2.0,13.0,5.0
2,Aaron Hicks-BAL,13.0,9.0,247.0,141.0
3,Aaron Hicks-NYY,11.0,6.0,112.0,53.0
4,Aaron Judge-NYY,34.0,21.0,459.0,190.0


In [49]:
df_runs = df_merged.groupby(by=['player_name','correct_type'])['missed_call_delta_run_exp'].sum().to_frame().reset_index().pivot_table('missed_call_delta_run_exp','player_name','correct_type').reset_index()

In [50]:
df_runs.rename(columns={'B':'missed_call_dre_B','S':'missed_call_dre_S'},inplace=True)

In [51]:
df_runs.head()

correct_type,player_name,missed_call_dre_B,missed_call_dre_S
0,A.J. Pollock-SEA,-1.096,0.934
1,A.J. Pollock-SF,-0.075,0.149
2,Aaron Hicks-BAL,-1.583,1.219
3,Aaron Hicks-NYY,-1.664,0.627
4,Aaron Judge-NYY,-4.257,2.557


In [52]:
df_missed_and_total_plus_runs = df_missed_and_total.merge(df_runs,left_on='player_name',right_on='player_name')

In [53]:
df_missed_and_total_plus_runs.head()

,player_name,B,S,ball,called_strike,missed_call_dre_B,missed_call_dre_S
0,A.J. Pollock-SEA,9.0,7.0,169.0,73.0,-1.096,0.934
1,A.J. Pollock-SF,1.0,2.0,13.0,5.0,-0.075,0.149
2,Aaron Hicks-BAL,13.0,9.0,247.0,141.0,-1.583,1.219
3,Aaron Hicks-NYY,11.0,6.0,112.0,53.0,-1.664,0.627
4,Aaron Judge-NYY,34.0,21.0,459.0,190.0,-4.257,2.557


In [54]:
df_batter_team = df[['player_name','batter_team']]

In [55]:
df_batter_team = df_batter_team.drop_duplicates()

In [56]:
df_mlb_pivoted = df_mlb_pivoted.merge(df_batter_team,how='left',left_on='player_name',right_on='player_name')

In [57]:
df_mlb_pivoted.head(25)

,player_name,B,S,batter_team
0,A.J. Pollock-SEA,9.0,7.0,SEA
1,A.J. Pollock-SF,1.0,2.0,SF
2,Aaron Hicks-BAL,13.0,9.0,BAL
3,Aaron Hicks-NYY,11.0,6.0,NYY
4,Aaron Judge-NYY,34.0,21.0,NYY
5,Abraham Almonte-NYM,2.0,1.0,NYM
6,Abraham Toro-MIL,2.0,0.0,MIL
7,Adam Duvall-BOS,25.0,10.0,BOS
8,Adam Frazier-BAL,35.0,19.0,BAL
9,Adam Haseley-CWS,3.0,5.0,CWS


In [58]:
df_missed_and_total_plus_runs = df_missed_and_total_plus_runs.merge(df_batter_team,how='left',left_on='player_name',right_on='player_name')

In [59]:
df_missed_and_total_plus_runs.head()

,player_name,B,S,ball,called_strike,missed_call_dre_B,missed_call_dre_S,batter_team
0,A.J. Pollock-SEA,9.0,7.0,169.0,73.0,-1.096,0.934,SEA
1,A.J. Pollock-SF,1.0,2.0,13.0,5.0,-0.075,0.149,SF
2,Aaron Hicks-BAL,13.0,9.0,247.0,141.0,-1.583,1.219,BAL
3,Aaron Hicks-NYY,11.0,6.0,112.0,53.0,-1.664,0.627,NYY
4,Aaron Judge-NYY,34.0,21.0,459.0,190.0,-4.257,2.557,NYY


In [60]:
df_mlb_pivoted.to_csv('mlb_calls_pivoted.csv')

In [61]:
df_missed_and_total_plus_runs.to_csv('mlb_missed_and_total_calls_plus_runs.csv')